In [2]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import os
import vectorbt as vbt
import io
import sys
from contextlib import redirect_stdout
from datetime import datetime
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

In [62]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')

# Machine learning imports
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from xgboost import XGBRegressor
from scipy.stats import uniform, randint

# Deep learning imports - using keras directly instead of tensorflow.keras
try:
    import keras
    from keras.models import Sequential
    from keras.layers import LSTM, Dense, Dropout
    from keras.callbacks import EarlyStopping
    KERAS_AVAILABLE = True
except ImportError:
    KERAS_AVAILABLE = False
    print("Keras not available, LSTM model will be skipped")

# Optional wavelet transform
try:
    import pywt
    WAVELETS_AVAILABLE = True
except ImportError:
    WAVELETS_AVAILABLE = False
    print("PyWavelets not available, wavelet features will be skipped")

Keras not available, LSTM model will be skipped
PyWavelets not available, wavelet features will be skipped


In [64]:
# Feature Engineering Functions

def calculate_rsi(series, window=14):
    """Calculate Relative Strength Index"""
    print(f"  Calculating RSI with {window}-day window for {series.name}")
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss.replace(0, 1e-9)  # Avoid division by zero
    rsi = 100 - (100 / (1 + rs))
    return rsi

def create_enhanced_features(df):
    print("\n" + "="*80)
    print("### ENHANCED FEATURE ENGINEERING ###")
    print("="*80)
    original_cols = df.columns.tolist()
    feature_count = len(df.columns)

    # 1.1 Multi-timeframe features - expanded range
    for window in [3, 5, 10, 15, 20, 30, 60, 90, 120, 250]:
        print(f"Creating {window}-day features...")

        # Moving averages of key indicators
        for col in ['us_ig_oas', 'us_hy_oas', 'cad_oas', 'vix']:
            if col in df.columns:
                df[f'{col}_ma{window}'] = df[col].rolling(window).mean()

        # Momentum indicators
        for col in ['us_ig_oas', 'us_hy_oas', 'cad_oas', 'cad_ig_er_index', 'us_ig_er_index']:
            if col in df.columns:
                df[f'{col}_mom{window}'] = df[col].pct_change(window)

        # Volatility features
        if 'cad_ig_er_index' in df.columns:
            df[f'er_vol_{window}d'] = df['cad_ig_er_index'].pct_change().rolling(window).std()
        if 'us_ig_oas' in df.columns:
            df[f'oas_vol_{window}d'] = df['us_ig_oas'].pct_change().rolling(window).std()

        # Spread ratios and differences
        if window >= 10:
            if all(col in df.columns for col in ['us_hy_oas', 'us_ig_oas']):
                ma_cols = [f'{col}_ma{window}' for col in ['us_hy_oas', 'us_ig_oas']]
                if all(col in df.columns for col in ma_cols):
                    df[f'hy_ig_oas_ratio_{window}d'] = df[f'us_hy_oas_ma{window}'] / df[f'us_ig_oas_ma{window}']

            if all(col in df.columns for col in ['cad_oas', 'us_ig_oas']):
                ma_cols = [f'{col}_ma{window}' for col in ['cad_oas', 'us_ig_oas']]
                if all(col in df.columns for col in ma_cols):
                    df[f'cad_us_oas_diff_{window}d'] = df[f'cad_oas_ma{window}'] - df[f'us_ig_oas_ma{window}']

    # 1.2 Rate environment features
    print("Creating enhanced rate environment features...")
    if 'us_3m_10y' in df.columns:
        df['curve_slope'] = df['us_3m_10y']
        df['curve_slope_change_20d'] = df['curve_slope'].diff(20)
        df['curve_slope_change_60d'] = df['curve_slope'].diff(60)

        # Slope momentum and acceleration
        df['curve_slope_mom'] = df['curve_slope'].pct_change(20)
        df['curve_slope_accel'] = df['curve_slope_mom'].diff(20)

        # Curve regimes
        df['flat_curve_regime'] = (df['curve_slope'].abs() < 0.5).astype(int)
        df['steep_curve_regime'] = (df['curve_slope'] > 1.0).astype(int)
        df['inverted_curve_regime'] = (df['curve_slope'] < 0).astype(int)

    # 1.3 Market regime indicators
    print("Creating market regime indicators...")
    # Volatility regimes
    if 'vix' in df.columns:
        df['vix_ma60'] = df['vix'].rolling(60).mean()
        df['high_vol_regime'] = (df['vix'] > df['vix_ma60']).astype(int)
        df['extreme_vol_regime'] = (df['vix'] > df['vix_ma60'] * 1.5).astype(int)
        df['vol_momentum'] = (df['vix'].pct_change(20) > 0).astype(int)

    # Credit spread regimes
    if 'us_ig_oas' in df.columns:
        df['ig_oas_ma60'] = df['us_ig_oas'].rolling(60).mean()
        df['widening_spreads'] = (df['us_ig_oas'] > df['ig_oas_ma60']).astype(int)
        df['oas_momentum'] = (df['us_ig_oas'].pct_change(20) > 0).astype(int)

    # Combined regimes
    if all(col in df.columns for col in ['high_vol_regime', 'widening_spreads']):
        df['vol_spread_regime'] = df['high_vol_regime'] * df['widening_spreads']

    if all(col in df.columns for col in ['inverted_curve_regime', 'high_vol_regime']):
        df['yield_vol_regime'] = df['inverted_curve_regime'] * df['high_vol_regime']

    # 1.4 Technical indicators
    print("Creating technical indicators...")
    # RSI for multiple timeframes
    for window in [14, 30, 60]:
        if 'cad_ig_er_index' in df.columns:
            df[f'cad_er_rsi_{window}'] = calculate_rsi(df['cad_ig_er_index'], window)
        if 'us_ig_er_index' in df.columns:
            df[f'us_er_rsi_{window}'] = calculate_rsi(df['us_ig_er_index'], window)
        if 'us_ig_oas' in df.columns:
            df[f'oas_rsi_{window}'] = calculate_rsi(df['us_ig_oas'], window)

    # 1.5 Lagged features
    print("Creating lagged relationships...")
    for lag in [1, 2, 3, 5]:
        for col in ['us_ig_oas', 'vix', 'us_ig_er_index', 'cad_ig_er_index']:
            if col in df.columns:
                df[f'{col}_lag{lag}'] = df[col].shift(lag)
                df[f'{col}_pct_lag{lag}'] = df[col].pct_change(lag).shift(1)

    # 1.6 Non-linear transformations and interactions
    print("Creating non-linear transformations and interactions...")
    for col in ['us_ig_oas', 'us_hy_oas', 'cad_oas', 'vix']:
        if col in df.columns and df[col].min() > 0:
            df[f'{col}_log'] = np.log(df[col])
            df[f'{col}_sqrt'] = np.sqrt(df[col])
            df[f'{col}_squared'] = df[col] ** 2

    # Cross-asset interactions
    if all(col in df.columns for col in ['us_ig_oas', 'vix']):
        df['oas_vix_interaction'] = df['us_ig_oas'] * df['vix']

    if all(col in df.columns for col in ['us_ig_oas', 'curve_slope']):
        df['oas_slope_interaction'] = df['us_ig_oas'] * df['curve_slope']

    if all(col in df.columns for col in ['vix', 'curve_slope']):
        df['vix_slope_interaction'] = df['vix'] * df['curve_slope']

    # Recent trend changes vs longer-term trends
    if all(col in df.columns for col in ['us_ig_oas_mom10', 'us_ig_oas_mom60']):
        df['oas_trend_divergence'] = df['us_ig_oas_mom10'] - df['us_ig_oas_mom60']

    if all(col in df.columns for col in ['cad_ig_er_index_mom10', 'cad_ig_er_index_mom60']):
        df['er_trend_divergence'] = df['cad_ig_er_index_mom10'] - df['cad_ig_er_index_mom60']

    # 1.7 Z-scores and normalized indicators
    print("Creating z-score features...")
    for col in ['us_ig_oas', 'us_hy_oas', 'vix', 'curve_slope']:
        if col in df.columns:
            df[f'{col}_zscore_60d'] = (df[col] - df[col].rolling(60).mean()) / df[col].rolling(60).std().replace(0, 1e-9)
            df[f'{col}_zscore_250d'] = (df[col] - df[col].rolling(250).mean()) / df[col].rolling(250).std().replace(0, 1e-9)

    # 1.8 Economic indicator interactions (if available)
    if 'us_economic_regime' in df.columns:
        print("Creating economic regime interactions...")
        if 'us_ig_oas' in df.columns:
            df['regime_oas'] = df['us_economic_regime'] * df['us_ig_oas']

        if 'curve_slope' in df.columns:
            df['regime_curve'] = df['us_economic_regime'] * df['curve_slope']

        if 'us_growth_surprises' in df.columns and 'us_ig_oas' in df.columns:
            df['growth_surprise_oas'] = df['us_growth_surprises'] * df['us_ig_oas']

            if 'vix' in df.columns:
                df['surprise_vol_interaction'] = df['us_growth_surprises'] * df['vix']

        if 'us_inflation_surprises' in df.columns and 'us_ig_oas' in df.columns:
            df['inflation_surprise_oas'] = df['us_inflation_surprises'] * df['us_ig_oas']

            if 'curve_slope' in df.columns:
                df['inflation_curve_interaction'] = df['us_inflation_surprises'] * df['curve_slope']

    # 1.9 NEW: Autocorrelation features
    print("Creating autocorrelation features...")
    for col in ['us_ig_oas', 'us_hy_oas', 'cad_oas', 'cad_ig_er_index']:
        if col in df.columns:
            for lag in [5, 10, 20]:
                # Calculate autocorrelation on rolling window
                try:
                    df[f'{col}_autocorr_{lag}'] = df[col].rolling(window=lag*2).apply(
                        lambda x: x.autocorr(lag=lag) if len(x.dropna()) > lag else np.nan,
                        raw=False
                    )
                    print(f"  Created autocorrelation feature: {col}_autocorr_{lag}")
                except Exception as e:
                    print(f"  Error creating autocorrelation for {col}_autocorr_{lag}: {e}")

    # 1.10 NEW: Exponentially weighted features
    print("Creating exponentially weighted features...")
    for span in [10, 30, 60]:
        for col in ['us_ig_oas', 'us_hy_oas', 'vix', 'curve_slope']:
            if col in df.columns:
                # EWM means and standard deviations
                df[f'{col}_ewm{span}'] = df[col].ewm(span=span).mean()
                df[f'{col}_ewm{span}_std'] = df[col].ewm(span=span).std()

                # EWM-based signals
                if span == 10:  # Only for the fast EWM
                    df[f'{col}_ewm_signal'] = (df[col] > df[f'{col}_ewm{span}']).astype(float)

        # Cross-EWM signals (fast vs slow)
        if span == 60:  # Only for the slow EWM
            for col in ['us_ig_oas', 'us_hy_oas', 'vix', 'curve_slope']:
                if col in df.columns and f'{col}_ewm10' in df.columns and f'{col}_ewm{span}' in df.columns:
                    # Crossover signals
                    df[f'{col}_ewm_crossover'] = (df[f'{col}_ewm10'] > df[f'{col}_ewm{span}']).astype(float)

                    # Divergence strength
                    df[f'{col}_ewm_divergence'] = (
                        (df[f'{col}_ewm10'] - df[f'{col}_ewm{span}']) / df[f'{col}_ewm{span}']
                    )

    # 1.11 NEW: Wavelet transform features (if available)
    if WAVELETS_AVAILABLE:
        print("Creating wavelet transform features...")
        for col in ['us_ig_oas', 'vix', 'cad_ig_er_index']:
            if col in df.columns:
                # Ensure no NaNs for wavelet transform
                series = df[col].fillna(method='ffill').fillna(method='bfill').values

                # Apply wavelet transform
                try:
                    # Use db4 wavelet with 3 levels of decomposition
                    coeffs = pywt.wavedec(series, 'db4', level=3)

                    # Reconstruct approximation and detail coefficients
                    for i, coef in enumerate(coeffs):
                        if i == 0:
                            # Approximation coefficients at the highest level
                            df[f'{col}_wavelet_a{i}'] = pywt.upcoef('a', coef, 'db4', level=3, take=len(series))
                        else:
                            # Detail coefficients at each level
                            df[f'{col}_wavelet_d{i}'] = pywt.upcoef('d', coef, 'db4', level=4-i, take=len(series))

                    print(f"  Created wavelet features for {col}")
                except Exception as e:
                    print(f"  Warning: Wavelet transform failed for {col}: {e}")

    new_features = len(df.columns) - feature_count
    print(f"\nFeature engineering complete: {new_features} new features created")
    print(f"Original features: {feature_count}, Total features now: {len(df.columns)}")
    return df

def create_return_targets(df):
    """Create return target variables for multiple horizons"""
    print("\n" + "="*80)
    print("### TARGET ENGINEERING ###")
    print("="*80)
    print("Creating return target variables...")

    # Multiple horizon returns
    for horizon in [1, 3, 5, 10, 20, 30, 60]:
        # Forward returns
        if 'cad_ig_er_index' in df.columns:
            df[f'target_return_{horizon}d'] = df['cad_ig_er_index'].pct_change(periods=horizon).shift(-horizon)

            # Log statistics
            valid_data = df[f'target_return_{horizon}d'].dropna()
            print(f"  Target {horizon}-day return:")
            print(f"    Mean: {valid_data.mean():.6f}")
            print(f"    Std Dev: {valid_data.std():.6f}")
            print(f"    Min: {valid_data.min():.6f}")
            print(f"    Max: {valid_data.max():.6f}")
            print(f"    Positive returns: {valid_data.gt(0).mean()*100:.1f}%")
            print(f"    Negative returns: {valid_data.lt(0).mean()*100:.1f}%")
        else:
            print(f"ERROR: Cannot create return targets - 'cad_ig_er_index' not found in dataframe")

    # Target correlations
    return_targets = [col for col in df.columns if col.startswith('target_return_')]
    if len(return_targets) > 1:
        target_corr = df[return_targets].corr()
        print("\nTarget return correlation matrix:")
        print(target_corr.round(2))

        # Print correlation between short and long-term returns
        if 'target_return_5d' in df.columns and 'target_return_20d' in df.columns:
            corr_5d_20d = df['target_return_5d'].corr(df['target_return_20d'])
            print(f"\nCorrelation between 5-day and 20-day returns: {corr_5d_20d:.4f}")

        if 'target_return_5d' in df.columns and 'target_return_60d' in df.columns:
            corr_5d_60d = df['target_return_5d'].corr(df['target_return_60d'])
            print(f"Correlation between 5-day and 60-day returns: {corr_5d_60d:.4f}")

    return df

def stable_feature_selection(X, y, n_iterations=50, sample_fraction=0.75, top_n=40):
    """
    Stability-based feature selection using multiple iterations of Lasso
    with subsampling for more robust selection.
    """
    print(f"Performing stability-based feature selection with {n_iterations} iterations...")
    print(f"  Sample fraction: {sample_fraction}, Target top features: {top_n}")
    feature_importance = np.zeros(X.shape[1])

    # Track both feature importance and selection frequency
    selection_freq = np.zeros(X.shape[1])

    for i in range(n_iterations):
        if i % 10 == 0 or i == n_iterations - 1:
            print(f"  Iteration {i+1}/{n_iterations}...")

        # Sample data
        sample_idx = np.random.choice(
            np.arange(X.shape[0]),
            size=int(X.shape[0] * sample_fraction),
            replace=False
        )
        X_sample, y_sample = X.iloc[sample_idx], y.iloc[sample_idx]

        # Fit model - try both Lasso and Random Forest for diversity
        if i % 2 == 0:
            # Lasso with cross-validation
            try:
                model = LassoCV(cv=5, random_state=i, max_iter=10000).fit(X_sample, y_sample)
                importance = np.abs(model.coef_)
                selected = importance > 0
                print(f"    Lasso selected {selected.sum()} features")
            except Exception as e:
                print(f"    Error in Lasso: {e}")
                # Fallback to Ridge which always works
                model = Ridge(alpha=1.0, random_state=i).fit(X_sample, y_sample)
                importance = np.abs(model.coef_)
                # Select top 30% of features
                threshold = np.percentile(importance, 70)
                selected = importance > threshold
                print(f"    Fallback to Ridge - selected {selected.sum()} features")
        else:
            # Random Forest
            try:
                model = RandomForestRegressor(
                    n_estimators=100, max_depth=5, random_state=i).fit(X_sample, y_sample)
                importance = model.feature_importances_
                # Select top 50% of features from RF
                threshold = np.percentile(importance, 50)
                selected = importance > threshold
                print(f"    Random Forest selected {selected.sum()} features")
            except Exception as e:
                print(f"    Error in Random Forest: {e}")
                # Fallback to Ridge
                model = Ridge(alpha=1.0, random_state=i).fit(X_sample, y_sample)
                importance = np.abs(model.coef_)
                threshold = np.percentile(importance, 70)
                selected = importance > threshold
                print(f"    Fallback to Ridge - selected {selected.sum()} features")

        # Update importance and frequency
        feature_importance += importance / n_iterations
        selection_freq += selected / n_iterations

    # Combine importance and frequency scores
    combined_score = feature_importance * np.sqrt(selection_freq)

    # Create feature ranking
    feature_ranks = pd.Series(combined_score, index=X.columns).sort_values(ascending=False)

    # Print selection frequency statistics
    selection_freq_series = pd.Series(selection_freq, index=X.columns)
    selection_stats = selection_freq_series.describe()
    print("\nFeature selection frequency statistics:")
    print(f"  Mean selection rate: {selection_stats['mean']:.4f}")
    print(f"  Min selection rate: {selection_stats['min']:.4f}")
    print(f"  Max selection rate: {selection_stats['max']:.4f}")
    print(f"  25th percentile: {selection_stats['25%']:.4f}")
    print(f"  50th percentile: {selection_stats['50%']:.4f}")
    print(f"  75th percentile: {selection_stats['75%']:.4f}")

    # Print top features
    print("\nTop 15 selected features:")
    for i, (feature, score) in enumerate(feature_ranks.head(15).items()):
        freq = selection_freq[X.columns.get_loc(feature)] * 100
        print(f"  {i+1}. {feature}: score={score:.6f}, selected in {freq:.1f}% of iterations")

    return feature_ranks.head(top_n).index.tolist()

In [65]:
# Regime Identification

def identify_improved_regimes(df, n_regimes=3):
    """
    Identify market regimes using Gaussian Mixture Model
    with smoothed transitions between regimes.
    """
    print("Identifying market regimes using Gaussian Mixture Model...")
    print(f"  Number of regimes to identify: {n_regimes}")

    # 1. Create regime-specific features
    regime_features = pd.DataFrame(index=df.index)

    # Extract key indicators and their rates of change
    key_indicators = ['vix', 'us_ig_oas', 'curve_slope']
    windows = [20, 60]

    # Calculate moving averages and momentum for key indicators
    for indicator in key_indicators:
        if indicator in df.columns:
            # Moving averages for different timescales
            for window in windows:
                regime_features[f'{indicator}_ma{window}'] = df[indicator].rolling(window).mean()
                print(f"  Created feature: {indicator}_ma{window}")

            # Add momentum indicators
            regime_features[f'{indicator}_mom20'] = df[indicator].pct_change(20)
            print(f"  Created feature: {indicator}_mom20")

            # Add volatility
            regime_features[f'{indicator}_vol20'] = df[indicator].rolling(20).std()
            print(f"  Created feature: {indicator}_vol20")

    # Add specific regime indicators
    if 'curve_slope' in df.columns:
        # Curve steepness/flatness
        regime_features['curve_steepness'] = df['curve_slope']
        print("  Added curve steepness feature")

    # Risk aversion indicator - simplified calculation to avoid indentation issues
    if 'vix' in df.columns and 'us_ig_oas' in df.columns:
        vix_ratio = df['vix'].rolling(30).mean() / df['vix'].rolling(252).mean()
        oas_ratio = df['us_ig_oas'].rolling(30).mean() / df['us_ig_oas'].rolling(252).mean()
        regime_features['risk_aversion'] = (vix_ratio + oas_ratio) / 2
        print("  Added risk aversion feature")

    # Fill missing values for clustering
    regime_features = regime_features.fillna(method='bfill').fillna(method='ffill')
    print(f"  Created {len(regime_features.columns)} features for regime identification")

    # 2. Apply dimensionality reduction to regime features
    print("Applying PCA for dimensionality reduction...")
    regime_features_clean = regime_features.dropna()
    print(f"  {len(regime_features_clean)} valid samples for regime identification")

    # Standardize data
    scaler = StandardScaler()
    regime_data_scaled = scaler.fit_transform(regime_features_clean)

    # Apply PCA to reduce dimensionality
    n_components = min(5, regime_data_scaled.shape[1])
    pca = PCA(n_components=n_components)
    regime_data_pca = pca.fit_transform(regime_data_scaled)

    # Print explained variance
    explained_variance = pca.explained_variance_ratio_
    print(f"  PCA explained variance ratios: {explained_variance.round(3)}")
    print(f"  Total explained variance: {explained_variance.sum():.2%}")
    print(f"  Reduced dimensionality from {regime_data_scaled.shape[1]} to {n_components} features")

    # 3. Apply Gaussian Mixture Model for regime identification
    print(f"Fitting Gaussian Mixture Model with {n_regimes} components...")
    gmm = GaussianMixture(
        n_components=n_regimes,
        covariance_type='full',
        random_state=42,
        n_init=10
    )

    # Fit the model and predict regimes
    gmm_regimes = gmm.fit_predict(regime_data_pca)
    regime_probs = gmm.predict_proba(regime_data_pca)

    # Get BIC and AIC
    bic = gmm.bic(regime_data_pca)
    aic = gmm.aic(regime_data_pca)
    print(f"  Model fit statistics: BIC={bic:.2f}, AIC={aic:.2f}")

    # 4. Create results dataframe
    regime_df = pd.DataFrame(index=regime_features_clean.index)
    regime_df['market_regime'] = gmm_regimes

    # Add regime probabilities
    for i in range(n_regimes):
        regime_df[f'regime_{i}_prob'] = regime_probs[:, i]

    # Add regime labels based on characteristics
    regime_labels = {}

    # Determine regime labels based on average characteristics
    # Higher OAS/VIX = stressed, Lower OAS/VIX = favorable
    regime_stats = {}

    for i in range(n_regimes):
        regime_mask = (regime_df['market_regime'] == i)
        regime_indices = regime_df.index[regime_mask]

        print(f"\nAnalyzing characteristics of Regime {i}:")
        print(f"  Samples in regime: {regime_mask.sum()} ({regime_mask.mean()*100:.1f}%)")

        # Calculate average values
        vix_avg = df.loc[regime_indices, 'vix'].mean() if 'vix' in df.columns else 0
        oas_avg = df.loc[regime_indices, 'us_ig_oas'].mean() if 'us_ig_oas' in df.columns else 0
        curve_avg = df.loc[regime_indices, 'curve_slope'].mean() if 'curve_slope' in df.columns else 0

        regime_stats[i] = {
            'vix_avg': vix_avg,
            'oas_avg': oas_avg,
            'curve_avg': curve_avg,
            'count': regime_mask.sum(),
            'pct': regime_mask.mean()
        }

        print(f"  Average VIX: {vix_avg:.2f}")
        print(f"  Average OAS: {oas_avg:.2f}")
        print(f"  Average Curve Slope: {curve_avg:.2f}")

    # Sort regimes by stress level (OAS + VIX)
    stress_levels = {i: stats['oas_avg'] + stats['vix_avg']/10 for i, stats in regime_stats.items()}
    sorted_regimes = sorted(stress_levels.items(), key=lambda x: x[1])

    # Assign labels based on sorted stress
    regime_labels = {}
    regime_labels[sorted_regimes[0][0]] = 'favorable'
    regime_labels[sorted_regimes[-1][0]] = 'stressed'

    # Middle regimes get 'neutral' label
    for i in range(1, len(sorted_regimes) - 1):
        regime_labels[sorted_regimes[i][0]] = 'neutral'

    # Map numeric regimes to labels
    regime_df['regime_label'] = regime_df['market_regime'].map(regime_labels)

    print("\nRegime label mapping:")
    for i, label in regime_labels.items():
        print(f"  Regime {i} -> {label}")

    # 5. Add smoothed regime indicators
    # Calculate exponentially weighted probabilities for smoother transitions
    for i in range(n_regimes):
        regime_df[f'regime_{i}_prob_smooth'] = regime_df[f'regime_{i}_prob'].ewm(span=5).mean()

    # Calculate regime transition indicators
    regime_df['regime_change'] = regime_df['market_regime'].diff().abs() > 0
    regime_df['days_in_regime'] = regime_df['regime_change'].cumsum()
    regime_df['days_since_change'] = regime_df.groupby('days_in_regime')['regime_change'].cumcount()

    # 6. Get overall regime statistics
    print("\nOverall regime distribution:")
    regime_counts = regime_df['regime_label'].value_counts(normalize=True)
    for label, pct in regime_counts.items():
        print(f"  {label}: {pct*100:.1f}%")

    # 7. Fill regime date mapping for test/train splits
    # Fill missing dates with the most common regime
    most_common_regime = regime_df['market_regime'].mode()[0]
    all_regime_df = pd.DataFrame(index=df.index)
    all_regime_df['market_regime'] = most_common_regime
    all_regime_df.loc[regime_df.index, 'market_regime'] = regime_df['market_regime']
    all_regime_df['regime_label'] = all_regime_df['market_regime'].map(regime_labels)

    for i in range(n_regimes):
        if f'regime_{i}_prob' in regime_df.columns:
            all_regime_df[f'regime_{i}_prob'] = 0.0
            all_regime_df.loc[regime_df.index, f'regime_{i}_prob'] = regime_df[f'regime_{i}_prob']

    # Print information about filled values
    filled_count = len(all_regime_df) - len(regime_df)
    if filled_count > 0:
        print(f"\nFilled {filled_count} missing dates with most common regime ({most_common_regime})")

    return all_regime_df

In [66]:
# Model Building Functions

def build_traditional_models(X_train, y_train):
    """Build traditional regression models"""
    print("Training traditional models...")

    models = {}
    model_params = {
        'ridge': {
            'alpha': 1.0,
            'random_state': 42
        },
        'elastic': {
            'alpha': 0.01,
            'l1_ratio': 0.5,
            'max_iter': 10000,
            'random_state': 42
        },
        'rf': {
            'n_estimators': 200,
            'max_depth': 4,
            'min_samples_leaf': 10,
            'random_state': 42
        },
        'gbr': {
            'n_estimators': 100,
            'learning_rate': 0.01,
            'max_depth': 3,
            'subsample': 0.8,
            'random_state': 42
        },
        'xgb': {
            'n_estimators': 100,
            'learning_rate': 0.01,
            'max_depth': 3,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42
        }
    }

    # Ridge regression
    print("  Training Ridge regression...")
    models['ridge'] = Ridge(**model_params['ridge'])
    models['ridge'].fit(X_train, y_train)

    # Print coefficients
    ridge_coefs = pd.Series(models['ridge'].coef_, index=X_train.columns)
    top_ridge_coefs = ridge_coefs.abs().sort_values(ascending=False)
    print(f"    Top 5 Ridge coefficients (absolute):")
    for i, (feat, coef) in enumerate(top_ridge_coefs.head(5).items()):
        actual_coef = ridge_coefs[feat]
        print(f"      {feat}: {actual_coef:.6f}")

    # ElasticNet
    print("  Training ElasticNet...")
    models['elastic'] = ElasticNet(**model_params['elastic'])
    models['elastic'].fit(X_train, y_train)

    # Print non-zero coefficients
    elastic_coefs = pd.Series(models['elastic'].coef_, index=X_train.columns)
    nonzero_coefs = elastic_coefs[elastic_coefs != 0]
    print(f"    ElasticNet selected {len(nonzero_coefs)} non-zero features")
    if len(nonzero_coefs) > 0:
        top_elastic_coefs = nonzero_coefs.abs().sort_values(ascending=False)
        print(f"    Top 5 ElasticNet coefficients (absolute):")
        for i, (feat, coef) in enumerate(top_elastic_coefs.head(5).items()):
            actual_coef = elastic_coefs[feat]
            print(f"      {feat}: {actual_coef:.6f}")

    # Random Forest
    print("  Training Random Forest...")
    models['rf'] = RandomForestRegressor(**model_params['rf'])
    models['rf'].fit(X_train, y_train)

    # Print feature importance
    rf_importance = pd.Series(models['rf'].feature_importances_, index=X_train.columns)
    top_rf_importance = rf_importance.sort_values(ascending=False)
    print(f"    Top 5 Random Forest important features:")
    for i, (feat, imp) in enumerate(top_rf_importance.head(5).items()):
        print(f"      {feat}: {imp:.6f}")

    # Gradient Boosting
    print("  Training Gradient Boosting...")
    models['gbr'] = GradientBoostingRegressor(**model_params['gbr'])
    models['gbr'].fit(X_train, y_train)

    # Print feature importance
    gbr_importance = pd.Series(models['gbr'].feature_importances_, index=X_train.columns)
    top_gbr_importance = gbr_importance.sort_values(ascending=False)
    print(f"    Top 5 Gradient Boosting important features:")
    for i, (feat, imp) in enumerate(top_gbr_importance.head(5).items()):
        print(f"      {feat}: {imp:.6f}")

    # XGBoost
    print("  Training XGBoost...")
    models['xgb'] = XGBRegressor(**model_params['xgb'])
    models['xgb'].fit(X_train, y_train)

    # Print feature importance
    xgb_importance = pd.Series(models['xgb'].feature_importances_, index=X_train.columns)
    top_xgb_importance = xgb_importance.sort_values(ascending=False)
    print(f"    Top 5 XGBoost important features:")
    for i, (feat, imp) in enumerate(top_xgb_importance.head(5).items()):
        print(f"      {feat}: {imp:.6f}")

    print(f"  Trained {len(models)} traditional models")
    return models

def build_regime_aware_models(X_train, y_train, regimes_train):
    """Build regime-specific models"""
    print("Training regime-specific models...")

    regime_models = {}

    # For each regime, train specialized models
    for regime in regimes_train['regime_label'].dropna().unique():
        # Create mask for this regime
        regime_mask = (regimes_train['regime_label'] == regime)
        regime_count = regime_mask.sum()

        if regime_count >= 100:  # Only train if we have sufficient data
            print(f"  Training models for '{regime}' regime ({regime_count} samples)...")

            # Get regime-specific data
            X_regime = X_train[regime_mask]
            y_regime = y_train[regime_mask]

            # Ridge model for this regime
            print(f"    Training Ridge model for {regime} regime...")
            ridge_model = Ridge(alpha=1.0, random_state=42)
            ridge_model.fit(X_regime, y_regime)
            regime_models[f'ridge_regime_{regime}'] = ridge_model

            # Print top coefficients
            ridge_coefs = pd.Series(ridge_model.coef_, index=X_train.columns)
            top_ridge_coefs = ridge_coefs.abs().sort_values(ascending=False)
            print(f"      Top 3 Ridge coefficients for {regime} regime:")
            for i, (feat, coef) in enumerate(top_ridge_coefs.head(3).items()):
                actual_coef = ridge_coefs[feat]
                print(f"        {feat}: {actual_coef:.6f}")

            # Random Forest model for this regime
            print(f"    Training Random Forest model for {regime} regime...")
            rf_model = RandomForestRegressor(
                n_estimators=100,
                max_depth=3,
                min_samples_leaf=5,
                random_state=42
            )
            rf_model.fit(X_regime, y_regime)
            regime_models[f'rf_regime_{regime}'] = rf_model

            # Print feature importance
            rf_importance = pd.Series(rf_model.feature_importances_, index=X_train.columns)
            top_rf_importance = rf_importance.sort_values(ascending=False)
            print(f"      Top 3 Random Forest important features for {regime} regime:")
            for i, (feat, imp) in enumerate(top_rf_importance.head(3).items()):
                print(f"        {feat}: {imp:.6f}")
        else:
            print(f"  Skipping '{regime}' regime - insufficient samples ({regime_count} < 100)")

    print(f"  Trained {len(regime_models)} regime-specific models")
    return regime_models

def build_advanced_models(X_train, y_train):
    """Build advanced ensemble and neural network models"""
    print("Training advanced models...")

    advanced_models = {}

    # 1. Stacking ensemble
    print("  Training stacking ensemble model...")
    base_models = [
        ('ridge', Ridge(alpha=1.0, random_state=42)),
        ('elastic', ElasticNet(alpha=0.01, l1_ratio=0.5, max_iter=10000, random_state=42)),
        ('rf', RandomForestRegressor(n_estimators=50, max_depth=3, random_state=42)),
        ('gbr', GradientBoostingRegressor(n_estimators=50, learning_rate=0.01, random_state=42))
    ]

    meta_learner = Ridge(alpha=1.0)

    stacking_model = StackingRegressor(
        estimators=base_models,
        final_estimator=meta_learner,
        cv=5,
        n_jobs=-1
    )

    try:
        stacking_model.fit(X_train, y_train)
        advanced_models['stacking'] = stacking_model
        print("    Stacking ensemble model trained successfully")

        # Try to extract feature importance from the meta-learner
        try:
            if hasattr(meta_learner, 'coef_'):
                meta_coefs = meta_learner.coef_
                print(f"    Meta-learner coefficients: {meta_coefs}")
        except:
            pass

    except Exception as e:
        print(f"    Error training stacking model: {e}")

    # 2. Neural network model
    print("  Training neural network model...")
    nn_model = MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        alpha=0.0001,
        batch_size=32,
        learning_rate='adaptive',
        max_iter=1000,
        early_stopping=True,
        validation_fraction=0.1,
        random_state=42,
        verbose=True
    )

    try:
        nn_model.fit(X_train, y_train)
        advanced_models['nn'] = nn_model
        print(f"    Neural network model trained successfully")
        print(f"    Training iterations: {nn_model.n_iter_}")
        print(f"    Final loss: {nn_model.loss_:.6f}")
    except Exception as e:
        print(f"    Error training neural network model: {e}")

    # 3. SVR model
    print("  Training SVR model...")
    svr_model = SVR(
        kernel='rbf',
        C=1.0,
        epsilon=0.1,
        gamma='scale'
    )

    try:
        svr_model.fit(X_train, y_train)
        advanced_models['svr'] = svr_model
        print("    SVR model trained successfully")
        print(f"    Support vectors: {svr_model.support_vectors_.shape[0]}")
    except Exception as e:
        print(f"    Error training SVR model: {e}")

    print(f"  Trained {len(advanced_models)} advanced models")
    return advanced_models

def build_lstm_model(X_train, y_train, lookback=20):
    """Build and train LSTM model for sequence prediction"""
    if not KERAS_AVAILABLE:
        print("Keras not available, skipping LSTM model")
        return None, None

    print(f"Training LSTM model with {lookback}-day lookback window...")

    # Convert to numpy arrays
    X_train_values = X_train.values
    y_train_values = y_train.values

    # Check if we have enough data
    if len(X_train_values) <= lookback:
        print("  Not enough data for LSTM training")
        return None, None

    # Create sequences for LSTM
    X_sequences = []
    y_sequences = []

    for i in range(len(X_train_values) - lookback):
        X_sequences.append(X_train_values[i:i+lookback])
        y_sequences.append(y_train_values[i+lookback])

    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)

    print(f"  LSTM input shape: {X_sequences.shape}")
    print(f"  LSTM output shape: {y_sequences.shape}")

    # Define LSTM model
    print("  Defining LSTM architecture...")
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(lookback, X_train.shape[1])),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    # Compile
    model.compile(optimizer='adam', loss='mse')

    # Print model summary
    model.summary()

    # Train with early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Train model
    try:
        print("  Training LSTM model...")
        history = model.fit(
            X_sequences, y_sequences,
            epochs=100,
            batch_size=32,
            validation_split=0.2,
            callbacks=[early_stop],
            verbose=2
        )

        # Print training history
        print("\n  LSTM training history:")
        for i, (train_loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
            if i % 10 == 0 or i == len(history.history['loss']) - 1:
                print(f"    Epoch {i+1}: train_loss={train_loss:.6f}, val_loss={val_loss:.6f}")

        print("  LSTM model training complete")
        return model, history
    except Exception as e:
        print(f"  Error training LSTM model: {e}")
        return None, None

In [68]:
# Prediction and Evaluation Functions

def predict_with_lstm(model, X_test, lookback=20):
    """Generate predictions with LSTM model"""
    if model is None:
        return np.zeros(len(X_test))

    print(f"Generating predictions with LSTM model (lookback={lookback})...")

    # Convert to numpy
    X_test_values = X_test.values

    # Check if we have enough data
    if len(X_test_values) <= lookback:
        print("  Not enough test data for LSTM prediction")
        return np.zeros(len(X_test))

    # Create sequences for prediction
    X_sequences = []
    for i in range(len(X_test_values) - lookback):
        X_sequences.append(X_test_values[i:i+lookback])

    X_sequences = np.array(X_sequences)
    print(f"  Test sequences shape: {X_sequences.shape}")

    # Make predictions
    print("  Making LSTM predictions...")
    predictions = model.predict(X_sequences, verbose=1)

    # Print prediction statistics
    print(f"  Prediction statistics: mean={predictions.mean():.6f}, std={predictions.std():.6f}")
    print(f"  Prediction range: [{predictions.min():.6f}, {predictions.max():.6f}]")

    # Pad with zeros for the first lookback points
    padded_predictions = np.zeros(len(X_test))
    padded_predictions[lookback:] = predictions.flatten()

    return padded_predictions

def create_regime_ensemble_predictions(models, X_test, regimes_test):
    """Create ensemble predictions using regime-specific models"""
    print("Creating regime-based ensemble predictions...")

    # Initialize predictions
    regime_preds = np.zeros(len(X_test))
    used_regime_model = np.zeros(len(X_test), dtype=bool)

    # Get unique regimes
    unique_regimes = regimes_test['regime_label'].dropna().unique()
    print(f"  Test set contains {len(unique_regimes)} regimes: {', '.join(unique_regimes)}")

    # Track regime model usage
    regime_model_usage = {regime: 0 for regime in unique_regimes}
    total_samples = len(X_test)

    # For each test sample, apply the appropriate regime model if available
    for i, (idx, regime) in enumerate(zip(X_test.index, regimes_test['regime_label'])):
        if pd.isna(regime):
            continue

        # Try ridge model first
        ridge_regime_key = f'ridge_regime_{regime}'
        if ridge_regime_key in models:
            regime_preds[i] = models[ridge_regime_key].predict(X_test.iloc[[i]])[0]
            used_regime_model[i] = True
            regime_model_usage[regime] += 1
        # Try RF model next
        else:
            rf_regime_key = f'rf_regime_{regime}'
            if rf_regime_key in models:
                regime_preds[i] = models[rf_regime_key].predict(X_test.iloc[[i]])[0]
                used_regime_model[i] = True
                regime_model_usage[regime] += 1

    # Print regime model usage statistics
    print("  Regime model usage:")
    for regime, count in regime_model_usage.items():
        pct = (count / total_samples) * 100
        print(f"    {regime}: {count} samples ({pct:.1f}%)")

    # For samples where no regime model was available, use global ridge model
    global_model_count = (~used_regime_model).sum()
    if global_model_count > 0:
        print(f"  Using global Ridge model for {global_model_count} samples ({global_model_count/total_samples*100:.1f}%)")
        if 'ridge' in models:
            regime_preds[~used_regime_model] = models['ridge'].predict(X_test.iloc[~used_regime_model])
        else:
            print("  WARNING: No global Ridge model available")

    # Print prediction statistics
    print(f"  Prediction statistics: mean={regime_preds.mean():.6f}, std={regime_preds.std():.6f}")
    print(f"  Prediction range: [{regime_preds.min():.6f}, {regime_preds.max():.6f}]")

    return regime_preds

def create_adaptive_ensemble(all_predictions, y_test_dict, horizons, best_models):
    """Create adaptive multi-horizon ensemble with dynamic weighting"""
    print("Creating adaptive multi-horizon ensemble...")

    # Get first prediction length (they might differ due to LSTM)
    first_horizon = horizons[0]
    first_model = best_models[first_horizon]
    pred_length = len(all_predictions[first_horizon][first_model])

    # Initialize weights and predictions
    model_weights = {}
    total_weight = 0
    ensemble_preds = np.zeros(pred_length)

    # Calculate weights based on directional accuracy and horizon
    print("\nCalculating model weights based on performance:")
    for horizon in horizons:
        model_name = best_models[horizon]
        predictions = all_predictions[horizon][model_name]
        print(f"  Using {model_name} for {horizon}-day horizon")

        # Verification length (might be shorter for LSTM)
        verify_length = min(len(predictions), len(y_test_dict[horizon]))

        # Calculate directional accuracy (key metric for trading)
        dir_acc = np.mean(
            (y_test_dict[horizon][:verify_length] > 0) ==
            (predictions[:verify_length] > 0)
        )

        # Weight by directional accuracy and inverse horizon
        raw_weight = dir_acc * (1.0 / horizon)
        model_weights[f'{horizon}d_{model_name}'] = raw_weight
        total_weight += raw_weight

        print(f"    Directional accuracy: {dir_acc:.4f}")
        print(f"    Raw weight: {raw_weight:.4f}")

        # Add weighted prediction
        if verify_length < pred_length:
            # Pad predictions if needed
            padded_preds = np.zeros(pred_length)
            padded_preds[:verify_length] = predictions[:verify_length]
            ensemble_preds += padded_preds * raw_weight
            print(f"    Note: Padded predictions from length {verify_length} to {pred_length}")
        else:
            ensemble_preds += predictions * raw_weight

    # Normalize weights
    for key in model_weights:
        model_weights[key] /= total_weight

    # Normalize ensemble predictions
    ensemble_preds /= total_weight

    # Print weights
    print("\nFinal model weights in ensemble:")
    for model, weight in sorted(model_weights.items(), key=lambda x: x[1], reverse=True):
        print(f"  {model}: {weight:.4f} ({weight*100:.1f}%)")

    # Print prediction statistics
    print(f"\nEnsemble prediction statistics:")
    print(f"  Mean: {ensemble_preds.mean():.6f}")
    print(f"  Std Dev: {ensemble_preds.std():.6f}")
    print(f"  Range: [{ensemble_preds.min():.6f}, {ensemble_preds.max():.6f}]")
    print(f"  Positive predictions: {(ensemble_preds > 0).mean()*100:.1f}%")

    return ensemble_preds, model_weights

def evaluate_model_detailed(y_true, y_pred, model_name):
    """Comprehensive model evaluation with detailed metrics"""
    # Basic metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    # Directional accuracy
    dir_acc = np.mean((y_true > 0) == (y_pred > 0))

    # Separate directional accuracy for positive/negative actual returns
    pos_mask = y_true > 0
    neg_mask = y_true <= 0

    pos_dir_acc = np.mean((y_pred[pos_mask] > 0)) if np.any(pos_mask) else np.nan
    neg_dir_acc = np.mean((y_pred[neg_mask] <= 0)) if np.any(neg_mask) else np.nan

    # Spearman rank correlation
    spearman_corr, spearman_pval = spearmanr(y_true, y_pred)

    # Profit simulation (simple strategy: long when prediction > 0, short when < 0)
    signals = np.sign(y_pred)
    returns = signals * y_true

    # Profit factor: sum of positive returns divided by absolute sum of negative returns
    pos_returns = returns[returns > 0].sum()
    neg_returns = np.abs(returns[returns < 0].sum())
    profit_factor = pos_returns / neg_returns if neg_returns != 0 else np.inf

    # Sharpe ratio (simplified)
    mean_return = returns.mean()
    std_return = returns.std()
    sharpe = (mean_return / std_return) * np.sqrt(252) if std_return != 0 else 0

    # Win rate
    win_rate = np.mean(returns > 0)

    # Print detailed results
    print(f"\nDetailed evaluation for {model_name}:")
    print(f"  MSE: {mse:.8f}")
    print(f"  RMSE: {rmse:.6f}")
    print(f"  MAE: {mae:.6f}")
    print(f"  R²: {r2:.4f}")

    print(f"  Directional Accuracy: {dir_acc:.4f}")
    print(f"    - For actual positive returns: {pos_dir_acc:.4f}")
    print(f"    - For actual negative returns: {neg_dir_acc:.4f}")

    print(f"  Spearman Correlation: {spearman_corr:.4f} (p={spearman_pval:.4f})")

    print(f"  Trading Performance:")
    print(f"    - Win Rate: {win_rate:.4f}")
    print(f"    - Profit Factor: {profit_factor:.4f}")
    print(f"    - Sharpe Ratio: {sharpe:.4f}")
    print(f"    - Mean Return: {mean_return:.6f}")

    return {
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'mae': mae,
        'dir_acc': dir_acc,
        'pos_dir_acc': pos_dir_acc,
        'neg_dir_acc': neg_dir_acc,
        'spearman_corr': spearman_corr,
        'spearman_pval': spearman_pval,
        'profit_factor': profit_factor,
        'win_rate': win_rate,
        'sharpe': sharpe,
        'mean_return': mean_return
    }

def analyze_feature_importance(models, feature_names):
    """Analyze feature importance from multiple models"""
    if 'rf' in models:
        # Get feature importance from Random Forest
        rf_model = models['rf']
        rf_importance = pd.Series(rf_model.feature_importances_, index=feature_names).sort_values(ascending=False)

        print("\nTop 15 features from Random Forest:")
        for i, (feat, imp) in enumerate(rf_importance.head(15).items()):
            print(f"  {i+1}. {feat}: {imp:.4f}")

    if 'gbr' in models:
        # Get feature importance from Gradient Boosting
        gb_model = models['gbr']
        gb_importance = pd.Series(gb_model.feature_importances_, index=feature_names).sort_values(ascending=False)

        print("\nTop 15 features from Gradient Boosting:")
        for i, (feat, imp) in enumerate(gb_importance.head(15).items()):
            print(f"  {i+1}. {feat}: {imp:.4f}")

    if 'xgb' in models:
        # Get feature importance from XGBoost
        xgb_model = models['xgb']
        xgb_importance = pd.Series(xgb_model.feature_importances_, index=feature_names).sort_values(ascending=False)

        print("\nTop 15 features from XGBoost:")
        for i, (feat, imp) in enumerate(xgb_importance.head(15).items()):
            print(f"  {i+1}. {feat}: {imp:.4f}")

    if 'elastic' in models:
        # Get coefficients from ElasticNet
        en_model = models['elastic']
        en_coef = pd.Series(en_model.coef_, index=feature_names)
        en_importance = en_coef.abs().sort_values(ascending=False)

        print("\nTop 15 features from ElasticNet:")
        for i, (feat, imp) in enumerate(en_importance.head(15).items()):
            coef_value = en_coef[feat]
            print(f"  {i+1}. {feat}: {coef_value:.6f}")

    # If multiple models available, calculate consensus importance
    if set(['rf', 'gbr', 'xgb']).issubset(set(models.keys())):
        print("\nConsensus feature importance (top 15):")

        # Get scaled importance from each model
        rf_imp = pd.Series(models['rf'].feature_importances_, index=feature_names)
        rf_imp = rf_imp / rf_imp.sum()

        gb_imp = pd.Series(models['gbr'].feature_importances_, index=feature_names)
        gb_imp = gb_imp / gb_imp.sum()

        xgb_imp = pd.Series(models['xgb'].feature_importances_, index=feature_names)
        xgb_imp = xgb_imp / xgb_imp.sum()

        # Calculate consensus score
        consensus = (rf_imp + gb_imp + xgb_imp) / 3

        # Print top features by consensus
        for i, (feat, imp) in enumerate(consensus.sort_values(ascending=False).head(15).items()):
            rf_val = rf_imp[feat]
            gb_val = gb_imp[feat]
            xgb_val = xgb_imp[feat]
            print(f"  {i+1}. {feat}: {imp:.4f} (RF: {rf_val:.4f}, GB: {gb_val:.4f}, XGB: {xgb_val:.4f})")

In [72]:

def main():
    print("\n" + "="*80)
    print("### ENHANCED FIXED INCOME RETURN PREDICTION SYSTEM ###")
    print("="*80)

    # Load and preprocess data
    print("\n" + "="*80)
    print("### LOADING AND PREPROCESSING DATA ###")
    print("="*80)

    data_path = 'c:/Users/Eddy/Documents/backtrader/data_pipelines/backtest_data.csv'
    df = pd.read_csv(data_path, index_col=0, parse_dates=True)
    print(f"Original data shape: {df.shape}")
    print(f"Date range: {df.index.min()} to {df.index.max()}")
    print(f"Original columns: {', '.join(df.columns.tolist())}")

    # Check for missing values and basic data quality
    missing_values = df.isnull().sum()
    print(f"\nMissing values check:")
    if missing_values.sum() > 0:
        print(f"Columns with missing values: {missing_values[missing_values > 0]}")
    else:
        print("No missing values found in original data")

    # Handle missing values
    df = df.fillna(method='ffill').fillna(method='bfill')
    print("Missing values filled using forward-fill and backward-fill methods")

    # Enhanced feature engineering
    df = create_enhanced_features(df)

    # Create return targets
    df = create_return_targets(df)

    # Clean data
    print("\n" + "="*80)
    print("### DATA CLEANING ###")
    print("="*80)
    print(f"Shape before dropping NaN: {df.shape}")
    orig_count = len(df)
    df = df.dropna()
    dropped_count = orig_count - len(df)
    dropped_pct = (dropped_count / orig_count) * 100 if orig_count > 0 else 0
    print(f"Shape after dropping NaN: {df.shape} (dropped {dropped_count} rows, {dropped_pct:.2f}%)")

    # Define primary target and horizons
    horizons = [5, 10, 20, 60]  # Added 60-day for longer-term signals
    primary_horizon = 5
    primary_target = f'target_return_{primary_horizon}d'
    print(f"\nPrediction horizons: {horizons} days")
    print(f"Primary prediction horizon: {primary_horizon} days")

    # Define features (exclude targets)
    feature_cols = [col for col in df.columns if not col.startswith('target_')]
    X = df[feature_cols]
    print(f"\nTotal features available: {len(feature_cols)}")

    # Display sample of feature names
    print("Sample of features:")
    sample_features = feature_cols[:5] + feature_cols[-5:] if len(feature_cols) > 10 else feature_cols
    for i, feat in enumerate(sample_features):
        print(f"  {i+1}. {feat}")
    if len(feature_cols) > 10:
        print(f"  ... and {len(feature_cols) - 10} more features")

    # Get targets for all horizons
    y_dict = {horizon: df[f'target_return_{horizon}d'] for horizon in horizons}

    # Print target statistics
    print("\nTarget statistics:")
    for horizon, y in y_dict.items():
        print(f"  {horizon}-day returns: mean={y.mean():.6f}, std={y.std():.6f}, min={y.min():.6f}, max={y.max():.6f}")
        print(f"    Positive returns: {(y > 0).mean()*100:.2f}%")

    # Feature selection - using stability selection
    print("\n" + "="*80)
    print("### FEATURE SELECTION WITH STABILITY SELECTION ###")
    print("="*80)
    selected_features = stable_feature_selection(X, y_dict[primary_horizon], n_iterations=30, sample_fraction=0.8, top_n=50)
    X = X[selected_features]
    print(f"Selected {len(selected_features)} features using stability selection")

    # Print all selected features
    print("\nAll selected features:")
    for i, feat in enumerate(selected_features):
        print(f"  {i+1}. {feat}")

    # Train/test split - more recent data as test
    split_pct = 0.8
    split_index = int(split_pct * len(X))
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train_dict = {h: y.iloc[:split_index] for h, y in y_dict.items()}
    y_test_dict = {h: y.iloc[split_index:] for h, y in y_dict.items()}

    print(f"\nTraining set: {X_train.shape[0]} samples from {X_train.index.min()} to {X_train.index.max()}")
    print(f"Testing set: {X_test.shape[0]} samples from {X_test.index.min()} to {X_test.index.max()}")

    # Print target statistics for train/test
    print("\nTraining set target statistics:")
    for horizon, y_train in y_train_dict.items():
        print(f"  {horizon}-day returns: mean={y_train.mean():.6f}, std={y_train.std():.6f}")
        print(f"    Positive returns: {(y_train > 0).mean()*100:.2f}%")

    print("\nTest set target statistics:")
    for horizon, y_test in y_test_dict.items():
        print(f"  {horizon}-day returns: mean={y_test.mean():.6f}, std={y_test.std():.6f}")
        print(f"    Positive returns: {(y_test > 0).mean()*100:.2f}%")

    # Scale features
    print("\n" + "="*80)
    print("### FEATURE SCALING ###")
    print("="*80)
    print("Using RobustScaler for feature scaling (robust to outliers)")
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Print scaling statistics
    print("Scaling summary:")
    scale_sample = np.random.choice(range(X_train.shape[1]), min(5, X_train.shape[1]), replace=False)
    for i in scale_sample:
        col = X_train.columns[i]
        print(f"  {col}:")
        print(f"    Before scaling: mean={X_train.iloc[:, i].mean():.4f}, std={X_train.iloc[:, i].std():.4f}")
        print(f"    After scaling: mean={X_train_scaled[:, i].mean():.4f}, std={X_train_scaled[:, i].std():.4f}")

    # Convert back to DataFrames for easier handling
    X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

    # Identify market regimes
    print("\n" + "="*80)
    print("### IDENTIFYING MARKET REGIMES ###")
    print("="*80)
    regimes_df = identify_improved_regimes(df)
    regimes_train = regimes_df.loc[X_train.index]
    regimes_test = regimes_df.loc[X_test.index]

    # Print regime distribution
    print("\nTraining set regime distribution:")
    train_regime_counts = regimes_train['regime_label'].value_counts(normalize=True)
    for regime, count in train_regime_counts.items():
        print(f"  {regime}: {count*100:.2f}%")

    print("\nTest set regime distribution:")
    test_regime_counts = regimes_test['regime_label'].value_counts(normalize=True)
    for regime, count in test_regime_counts.items():
        print(f"  {regime}: {count*100:.2f}%")

    # Initialize storage for models and metrics
    all_models = {}
    all_predictions = {}
    model_metrics = {}
    best_models_by_horizon = {}
    best_metrics_by_horizon = {}

    # Build models for each horizon
    print("\n" + "="*80)
    print("### TRAINING MODELS FOR MULTIPLE HORIZONS ###")
    print("="*80)

    for horizon in horizons:
        print("\n" + "-"*70)
        print(f"## TRAINING MODELS FOR {horizon}-DAY HORIZON ##")
        print("-"*70)

        # Get target for this horizon
        y_train = y_train_dict[horizon]
        y_test = y_test_dict[horizon]

        # Build traditional models
        traditional_models = build_traditional_models(X_train_scaled_df, y_train)

        # Build regime-aware models
        regime_models = build_regime_aware_models(X_train_scaled_df, y_train, regimes_train)

        # Build advanced models
        advanced_models = build_advanced_models(X_train_scaled_df, y_train)

        # Store all models for this horizon
        all_models[horizon] = {**traditional_models, **regime_models, **advanced_models}

        # Generate and evaluate predictions
        horizon_predictions = {}

        print("\n" + "-"*70)
        print(f"## EVALUATING MODELS FOR {horizon}-DAY HORIZON ##")
        print("-"*70)

        best_model_name = None
        best_dir_acc = -1
        best_profit = -1

        for model_name, model in all_models[horizon].items():
            # Skip regime-specific models in direct evaluation
            if model_name.startswith(('ridge_regime_', 'rf_regime_')):
                continue

            # Generate predictions
            print(f"\nGenerating predictions for model: {model_name}")
            y_pred = model.predict(X_test_scaled_df)

            # Store predictions
            horizon_predictions[model_name] = y_pred

            # Evaluate model with detailed metrics
            metrics = evaluate_model_detailed(y_test, y_pred, f"{horizon}d {model_name}")
            model_metrics[(horizon, model_name)] = metrics

            # Track best model
            if metrics['dir_acc'] > best_dir_acc:
                best_dir_acc = metrics['dir_acc']
                best_model_name = model_name

            if metrics['profit_factor'] > best_profit:
                best_profit = metrics['profit_factor']

        # Create regime-weighted ensemble predictions
        if any(k.startswith('ridge_regime_') for k in all_models[horizon].keys()):
            print("\nGenerating regime-aware ensemble predictions")
            regime_ensemble_preds = create_regime_ensemble_predictions(
                all_models[horizon], X_test_scaled_df, regimes_test)
            horizon_predictions['regime_ensemble'] = regime_ensemble_preds

            # Evaluate regime ensemble
            metrics = evaluate_model_detailed(y_test, regime_ensemble_preds, f"{horizon}d regime_ensemble")
            model_metrics[(horizon, 'regime_ensemble')] = metrics

            # Check if this is the best model
            if metrics['dir_acc'] > best_dir_acc:
                best_dir_acc = metrics['dir_acc']
                best_model_name = 'regime_ensemble'

        # Create simple model ensemble (average predictions)
        print("\nGenerating model ensemble predictions (average of all models)")
        model_ensemble_preds = np.zeros(len(X_test))
        ensemble_count = 0

        for model_name, preds in horizon_predictions.items():
            if not model_name.startswith('regime'):  # Exclude regime ensemble
                model_ensemble_preds += preds
                ensemble_count += 1

        model_ensemble_preds /= ensemble_count
        horizon_predictions['model_ensemble'] = model_ensemble_preds

        # Evaluate model ensemble
        metrics = evaluate_model_detailed(y_test, model_ensemble_preds, f"{horizon}d model_ensemble")
        model_metrics[(horizon, 'model_ensemble')] = metrics

        # Check if this is the best model
        if metrics['dir_acc'] > best_dir_acc:
            best_dir_acc = metrics['dir_acc']
            best_model_name = 'model_ensemble'

        # Store all predictions for this horizon
        all_predictions[horizon] = horizon_predictions

        # Store best model for this horizon
        best_models_by_horizon[horizon] = best_model_name
        best_metrics_by_horizon[horizon] = model_metrics[(horizon, best_model_name)]

        print(f"\nBest model for {horizon}-day horizon: {best_model_name}")
        print(f"  Directional Accuracy: {best_dir_acc:.4f}")

    # Train LSTM model for sequence prediction (if Keras is available)
    if KERAS_AVAILABLE:
        print("\n" + "="*80)
        print("### TRAINING LSTM SEQUENCE MODEL ###")
        print("="*80)

        lstm_model, lstm_history = build_lstm_model(
            X_train_scaled_df, y_train_dict[primary_horizon], lookback=20)

        if lstm_model is not None:
            # Get LSTM predictions
            print("\nGenerating LSTM predictions")
            lstm_preds = predict_with_lstm(lstm_model, X_test_scaled_df, lookback=20)

            # Evaluate LSTM
            print("\nEvaluating LSTM model:")
            valid_indices = ~np.isnan(lstm_preds)
            if np.any(valid_indices):
                y_valid = y_test_dict[primary_horizon].iloc[valid_indices]
                lstm_valid_preds = lstm_preds[valid_indices]

                metrics = evaluate_model_detailed(
                    y_valid, lstm_valid_preds, f"{primary_horizon}d LSTM"
                )
                model_metrics[(primary_horizon, 'lstm')] = metrics

                # Store LSTM predictions
                all_predictions[primary_horizon]['lstm'] = lstm_preds

                # Check if LSTM is best for primary horizon
                if metrics['dir_acc'] > best_metrics_by_horizon[primary_horizon]['dir_acc']:
                    best_models_by_horizon[primary_horizon] = 'lstm'
                    best_metrics_by_horizon[primary_horizon] = metrics
                    print(f"LSTM is now the best model for {primary_horizon}-day horizon!")
            else:
                print("Could not evaluate LSTM - all predictions are NaN")
    else:
        print("\nSkipping LSTM model (Keras not available)")

    # Create adaptive multi-horizon ensemble
    print("\n" + "="*80)
    print("### CREATING ADAPTIVE MULTI-HORIZON ENSEMBLE ###")
    print("="*80)

    # Print best models for each horizon
    print("\nBest model for each horizon:")
    for horizon in horizons:
        best_model = best_models_by_horizon[horizon]
        best_metrics = best_metrics_by_horizon[horizon]
        print(f"  {horizon}-day horizon: {best_model}")
        print(f"    Directional Accuracy: {best_metrics['dir_acc']:.4f}")
        print(f"    R²: {best_metrics['r2']:.4f}")
        print(f"    Profit Factor: {best_metrics['profit_factor']:.4f}")

    # Create adaptive multi-horizon ensemble
    adaptive_ensemble_preds, model_weights = create_adaptive_ensemble(
        all_predictions, y_test_dict, horizons, best_models_by_horizon)

    # Print weights
    print("\nAdaptive ensemble weights:")
    for model, weight in sorted(model_weights.items(), key=lambda x: x[1], reverse=True):
        print(f"  {model}: {weight:.4f}")

    # Evaluate adaptive ensemble on primary horizon
    print("\nEvaluating adaptive multi-horizon ensemble:")
    metrics = evaluate_model_detailed(
        y_test_dict[primary_horizon],
        adaptive_ensemble_preds,
        "Adaptive multi-horizon ensemble"
    )
    model_metrics[('adaptive', 'ensemble')] = metrics

    # Feature importance analysis
    print("\n" + "="*80)
    print("### FEATURE IMPORTANCE ANALYSIS ###")
    print("="*80)

    analyze_feature_importance(all_models[primary_horizon], X_train.columns)

    # Print final performance summary
    print("\n" + "="*80)
    print("### FINAL PERFORMANCE SUMMARY ###")
    print("="*80)

    # Create summary table of all models for primary horizon
    print(f"\nPerformance metrics for {primary_horizon}-day horizon models:")
    print("-" * 100)
    print(f"{'Model':<20} {'Dir. Acc.':<10} {'R²':<10} {'RMSE':<10} {'MAE':<10} {'Profit Factor':<15} {'Sharpe':<10}")
    print("-" * 100)

    for model_key, metrics in model_metrics.items():
        # Only show primary horizon models in summary
        if model_key[0] == primary_horizon:
            model_name = model_key[1]
            print(f"{model_name:<20} {metrics['dir_acc']:<10.4f} {metrics['r2']:<10.4f} "
                  f"{metrics['rmse']:<10.6f} {metrics['mae']:<10.6f} {metrics['profit_factor']:<15.4f} "
                  f"{metrics['sharpe']:<10.4f}")

    # Add adaptive ensemble to table
    if ('adaptive', 'ensemble') in model_metrics:
        metrics = model_metrics[('adaptive', 'ensemble')]
        print(f"{'adaptive_ensemble':<20} {metrics['dir_acc']:<10.4f} {metrics['r2']:<10.4f} "
              f"{metrics['rmse']:<10.6f} {metrics['mae']:<10.6f} {metrics['profit_factor']:<15.4f} "
              f"{metrics['sharpe']:<10.4f}")

    print("-" * 100)

    # Directional accuracy comparison across horizons
    print("\nDirectional accuracy comparison across horizons:")
    print("-" * 70)

    # Fixed: Avoid nested f-string formatting issues
    horizon_headers = "".join([f"{h}-day".ljust(12) for h in horizons])
    print(f"{'Model':<20} {horizon_headers}")

    print("-" * 70)

    # Common models across all horizons
    common_models = ['ridge', 'elastic', 'rf', 'xgb', 'model_ensemble', 'regime_ensemble']

    for model in common_models:
        row = f"{model:<20} "
        for horizon in horizons:
            if (horizon, model) in model_metrics:
                row += f"{model_metrics[(horizon, model)]['dir_acc']:<12.4f}"
            else:
                row += f"{'N/A':<12}"
        print(row)

    print("-" * 70)

    # Print adaptive ensemble performance
    if ('adaptive', 'ensemble') in model_metrics:
        metrics = model_metrics[('adaptive', 'ensemble')]

        print("\nAdaptive Multi-Horizon Ensemble Final Performance:")
        print(f"R²: {metrics['r2']:.4f}")
        print(f"RMSE: {metrics['rmse']:.6f}")
        print(f"Directional Accuracy: {metrics['dir_acc']:.4f}")
        print(f"Profit Factor: {metrics['profit_factor']:.4f}")
        print(f"Sharpe Ratio: {metrics['sharpe']:.4f}")

    print("\n" + "="*80)
    print("### EXECUTION COMPLETE ###")
    print("="*80)

    return all_models, all_predictions, adaptive_ensemble_preds, model_metrics


In [73]:
# Execution 

if __name__ == "__main__":
    all_models, all_predictions, adaptive_ensemble_preds, model_metrics = main()


### ENHANCED FIXED INCOME RETURN PREDICTION SYSTEM ###

### LOADING AND PREPROCESSING DATA ###
Original data shape: (5902, 15)
Date range: 2002-10-31 00:00:00 to 2025-02-27 00:00:00
Original columns: cad_oas, us_hy_oas, us_ig_oas, tsx, vix, us_3m_10y, us_growth_surprises, us_inflation_surprises, us_lei_yoy, us_hard_data_surprises, us_equity_revisions, us_economic_regime, cad_ig_er_index, us_hy_er_index, us_ig_er_index

Missing values check:
No missing values found in original data
Missing values filled using forward-fill and backward-fill methods

### ENHANCED FEATURE ENGINEERING ###
Creating 3-day features...
Creating 5-day features...
Creating 10-day features...
Creating 15-day features...
Creating 20-day features...
Creating 30-day features...
Creating 60-day features...
Creating 90-day features...
Creating 120-day features...
Creating 250-day features...
Creating enhanced rate environment features...
Creating market regime indicators...
Creating technical indicators...
  Calculati